In [12]:
import os
import datetime
import json
import numpy as np
from keras.models import load_model
import pandas as pds

from time import sleep
#from ast import keyword
from encodings import search_function
from os import remove

import datetime
from re import search
import numpy as np
import sqlite3
import glob
import scipy
import gc
import pickle
import collections
import csv
import codecs

import string
import re
#import torch
#import datasets
import tensorflow as tf
import pandas as pds
import numpy as np
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, Callback, CSVLogger, ModelCheckpoint
print(tf.__version__) #tensorflowのver
from keras.utils import to_categorical


2.10.0


In [13]:
#作品データのディレクトリ
aozora_dir = r""
#作者のリスト
sakusya_list = os.listdir(aozora_dir)
print(sakusya_list)

['内田 魯庵', '南部 修太朗', '古川 緑波', '土田 耕平', '大倉 てる子', '宮原 晃一郎', '小栗 虫太郎', '山之口 獏', '新渡戸 稲造', '桑原 隲蔵', '正岡 容', '泉 鏡太郎', '渡辺 温', '竹久 夢二', '萩原 朔太郎', '葛西 善蔵', '蘭 郁二郎', '野上 豊一郎', '野口 雨情', '高浜 虚子']


In [5]:
def makeDir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [6]:
# Making txt files with SHIFT-JIS converted to UTF-8.

def convert_sjis_to_utf8(auth_target,now_dir,sakusya_name):
    #aaa.txt
    file_name = auth_target[0]
    #/now/aaa.txt
    file_dir = os.path.join(now_dir,file_name)
    #now/utf
    makeDir(now_dir + "/utf")
    utf_dir = '{}{}'.format(now_dir, "/utf")
    save_dir = utf_dir + "/" + file_name

    log_np = np.array([["datetime", "author", "file"]])
    utf_dir = '{}{}'.format(file_dir, "utf/")
    fout = codecs.open(file_dir, 'r', 'shift_jis')
    fsave = codecs.open(save_dir, 'w+', 'utf-8')
    try:
        for row in fout:
            fsave.write(row)
    except Exception as e:
        print ("gets exception: " + str(type(e)))
    finally:
        fout.close()
        fsave.close()
        now = "4/08"
        tmp = np.array([[now,sakusya_name, file_name]])
        log_np = np.vstack((log_np, tmp))              
    convert_pds = pds.DataFrame(log_np[1:,:], columns=log_np[0,:])
    #convert_pds.to_csv(now_dir + file_name + '_cvt_log.csv', quoting=csv.QUOTE_ALL)

In [8]:
def data_cleanse(auth_target,now_dir,sakusya_name):
    #aaa.txt
    file_name = auth_target[0]
    #/now/aaa.txt
    file_dir = os.path.join(now_dir,file_name)
    #print(file_dir)
    #now/utf
    makeDir(now_dir + "/utf")
    makeDir(now_dir + "/csv")
    utf_dir = '{}{}'.format(now_dir, "/utf")
    save_dir = utf_dir + "/" + file_name
    csv_dir = '{}{}'.format(now_dir, "/csv")

    np_lines = np.array([["author","piece","line"]])
    f = open(save_dir, 'r',encoding="utf-8")
    lines = f.read()
    f.close
    
    lines = lines.replace(u'。', '。\n')
    lines = lines.split('\n')

    ruby = re.compile(u'\《.+?\》')
    chuki = re.compile(u'\［.+?\］')
    zen_sp = re.compile(u'　')
    zen_sp2 = re.compile(u'\u3000')

    for line in lines:
        line_mod = ruby.sub("", line)
        line_mod = chuki.sub("", line_mod)
        line_mod = zen_sp.sub("", line_mod)
        line_mod = zen_sp2.sub("", line_mod)
        if len(line_mod) != 1:
            np_tmp = np.array([[sakusya_name, file_name, line_mod]])
            np_lines = np.vstack((np_lines, np_tmp))

    s_line = 1
    e_line = len(lines)
    np_lines_cut = np_lines[s_line:e_line,:]


    file = file_name.replace(".txt", "")
    lines_pds = pds.DataFrame(np_lines_cut, columns=np_lines[0,:])
    lines_pds.to_csv(csv_dir + "/" + file + '.csv',encoding="utf-8")

In [14]:
def author_data_integ():
    #作者ごとにlen(sakusya_list)
    for i in range(0,len(sakusya_list)):
        marge_csv = []
        sakusya_name = sakusya_list[i]
        date_path = ""
        date_path = os.path.join(aozora_dir, sakusya_name)
        date_path = os.path.join(date_path,"学習データ")
        date_list = os.listdir(date_path)
        #print(date_list)
        #テキストの数だけ
        integ_np = np.array([["author", "line"]])
        for file in date_list:
            if "integ.csv" in file:
                pass
            else:
                date_path2 = os.path.join(date_path,file)
                #学習データ/iii/csv
                csv_dir = os.path.join(date_path2,"csv")
                csv_file = os.listdir(csv_dir)
                csv_file2 = csv_file[0]
                if "txt" in csv_file:
                    csv_file2 = csv_file[1] 
                #integ_np = np.array([["author", "line"]])
                csv_file_dir = os.path.join(csv_dir,csv_file2)

                pds_data = pds.read_csv(csv_file_dir,index_col=0)
                pds_data = pds_data.dropna()
                np_data = np.array(pds_data.iloc[:,[0,2]])
                out = [j for j in range(len(np_data)) if '-----------' in str(np_data[j,1])]
                if not out: out = [1]
                hyphen_pos = int(out[len(out) - 1])

                last_20 = len(np_data) - 20

                np_data = np_data[hyphen_pos+1:last_20,:]
                integ_np = np.vstack((integ_np, np_data))
            #print(pds_data)
            #marge_csv.append(pds_data)
        integ_pds = pds.DataFrame(integ_np[1:,:], columns=integ_np[0,:])
        integ_pds.to_csv(date_path+ "/" + sakusya_name + '_integ.csv', quoting=csv.QUOTE_ALL)   
    
            #前5行と後ろ20行を削除
        #integ_pds = pds.concat(marge_csv, ignore_index=True)    
        #integ_pds.to_csv(date_path+ "/" + sakusya_name + '_integ.csv', quoting=csv.QUOTE_ALL)

In [15]:
# Loading integrated csv files.

def load_integ(author, auth_dir):
    integ_csv_list = os.listdir(auth_dir)
    for i in integ_csv_list:
        #print(i)
        if "csv" in i:
            integ_csv = i
    integ_csv_dir = os.path.join(auth_dir,integ_csv)
    data = pds.read_csv(integ_csv_dir)
    #print(data)
    return data

In [16]:
# Converting each line of texts to encoded array.

def load_data(txt, max_length=100):
    txt_list = []
    for l in txt:
        txt_line = [ord(x) for x in str(l).strip()]
        # You will get encoded text in array, just like this
        # [25991, 31456, 12391, 12399, 12394, 12367, 12387, 12390, 23383, 24341, 12391, 12354, 12427, 12290]
        #print(txt_line)
        txt_line = txt_line[:max_length]
        txt_len = len(txt_line)
        if txt_len < max_length:
            txt_line += ([0] * (max_length - txt_len))
        txt_list.append((txt_line))
    return txt_list

In [ ]:
target_author = []
for i in range(0,len(sakusya_list)):
    sakusya_name = sakusya_list[i]
    target_author.append(sakusya_name)
    print(sakusya_name)
    date_path = os.path.join(aozora_dir, sakusya_name)
    date_path = os.path.join(date_path,"学習データ")
    date_list = os.listdir(date_path)
    for j in range(0,len(date_list)):
        now_date = date_list[j]
        #print(now_date)
        if "integ.csv" not in now_date:
            #print(now_date)
            #学習データ/rubi.../
            date_path_2 = os.path.join(date_path,now_date)
            gakusyudate = os.listdir(date_path_2)
            #print(gakusyudate)
            convert_sjis_to_utf8(gakusyudate,date_path_2,sakusya_name)
            data_cleanse(gakusyudate,date_path_2,sakusya_name)
#print(gakusyudate)

In [37]:
author_data_integ()

In [38]:
#作者ごとに
sakusya_np_list = []
for i in range(0,len(sakusya_list)):
    marge_csv = []
    sakusya_name = sakusya_list[i]
    date_path = os.path.join(aozora_dir, sakusya_name)
    date_path = os.path.join(date_path,"学習データ")
    date_list = os.listdir(date_path)
    date = load_integ(sakusya_name,date_path)
    np_date = np.array(date.iloc[0:,1:])
    txt = np.array([np_date[:,1]]).T
    #print(txt)
    id = np.array([np.zeros(len(np_date)) + i]).T
    #print(id)
    sakusya = np.hstack((txt, id))
    #print(len(sakusya))
    sakusya_np_list.append(sakusya)
#print(len(sakusya_np_list))
    

In [39]:
print(sakusya_np_list[0])
print(sakusya_np_list[1])
print(sakusya_np_list[2])
print(sakusya_np_list[19])


[['占ひ、人相、方角、気にしだしたら際限が無い。' 0.0]
 ['朝から晩まで易者、人相、方位家と駈けずり廻つても追付かない。' 0.0]
 ['が、中には道楽で、何にも見て貰う事も無いのにソコラ中へ冷かしに行くものがある。' 0.0]
 ...
 ['小児の如くタワイなく、意気地なく、湾白で、ダヾをこねて、遊び好で、無法で、歿分暁で、或時はお山の大将となりて空威張をし、或時はデレリ茫然としてお芋の煮えたも御存じなきお目出たき者は当世の文学者を置いて誰ぞや。'
  0.0]
 ['文学者なる哉、文学者なる哉。' 0.0]
 ['天変地異を笑つて済ますものは文学者なり。' 0.0]]
[['十七の五月だつた。' 1.0]
 ['私は重い膓チブスに罹つて、赤坂の或る病院へ入院した。' 1.0]
 ['入院して十日餘りは私はまるで夢中だつた。' 1.0]
 ...
 ['と、Ｙは言はずにはゐられないと云つたやうな樣子で、いきなりその重い沈默を破つてしまひました。' 1.0]
 ['が、皆はそれに答へようともしないで、やつぱり沈默を續けてゐました。' 1.0]
 ['「だがねえ、君、僕の場合に於てさ、その時の女の心持つて一體どう云ふのだらう。' 1.0]]
[['よき日、よき頃のはなしである。' 2.0]
 ['フランスの汽船会社Ｍ・Ｍの船が、神戸の港へ入ると、その船へ昼食を食べに行くことが出来たものだった。' 2.0]
 ['はじめての時は、フェリックス・ルセルという船だった。' 2.0]
 ...
 ['然し此ういふことがあると勉強になる。' 2.0]
 ['世に愚かほど罪悪はない。' 2.0]
 ['そして、政治は嘘でやらなければダメだといふこと。' 2.0]]
[['松山城の北に練兵場がある。' 19.0]
 ['ある夏の夕｜其処へ行って当時中学生であった余らがバッチングを遣っていると、其処へぞろぞろと東京がえりの四、六人の書生が遣って来た。'
  19.0]
 ['余らも裾を短くし腰に手拭をはさんで一ぱし書生さんの積りでいたのであったが、その人々は本場仕込みのツンツルテンで脛の露出し具合もいなせなり腰にはさんだ手拭も赤い色のにじんだタオルなどであることがまず人目を欹たしめるのであった。'
  19.0]
 ...
 ['高崎山は四極山というそうである。' 19

In [19]:
#4人でのお試し用
date_integ = np.vstack((np.vstack((np.vstack((sakusya_np_list[0], sakusya_np_list[1])),sakusya_np_list[2])),sakusya_np_list[3]))

In [22]:
date_integ = np.vstack((sakusya_np_list[0],sakusya_np_list[1]))
for i in range(2,len(sakusya_np_list)):
    date_integ = np.vstack((date_integ,sakusya_np_list[i]))


In [23]:
print(date_integ,"\n")
print(date_integ[:,0])
print(date_integ[:,1])

[['占ひ、人相、方角、気にしだしたら際限が無い。' 0.0]
 ['朝から晩まで易者、人相、方位家と駈けずり廻つても追付かない。' 0.0]
 ['が、中には道楽で、何にも見て貰う事も無いのにソコラ中へ冷かしに行くものがある。' 0.0]
 ...
 ['高崎山は四極山というそうである。' 19.0]
 ['きのふは二百二十日であつて、おとゝひから蒸し暑く豪雨を降らしたり嵐がおとづれたりして、たゞならん景色であつた。' 19.0]
 ['それがきのふ午後になつて颱風が過ぎ去つて了つたのであらう、空を見ると雲は未だ空を覆うてはをるが併しその雲の色も險惡な色は無くなつて、殊に風の方向を示して居るやうな矢のやうな雲が數多く現れてその矢のやうな雲の外に綿のやうな雲が散らばつて居て、その綿のやうな雲の間から珍しくも青空が見えて來た。'
  19.0]] 

['占ひ、人相、方角、気にしだしたら際限が無い。' '朝から晩まで易者、人相、方位家と駈けずり廻つても追付かない。'
 'が、中には道楽で、何にも見て貰う事も無いのにソコラ中へ冷かしに行くものがある。' ... '高崎山は四極山というそうである。'
 'きのふは二百二十日であつて、おとゝひから蒸し暑く豪雨を降らしたり嵐がおとづれたりして、たゞならん景色であつた。'
 'それがきのふ午後になつて颱風が過ぎ去つて了つたのであらう、空を見ると雲は未だ空を覆うてはをるが併しその雲の色も險惡な色は無くなつて、殊に風の方向を示して居るやうな矢のやうな雲が數多く現れてその矢のやうな雲の外に綿のやうな雲が散らばつて居て、その綿のやうな雲の間から珍しくも青空が見えて來た。']
[0.0 0.0 0.0 ... 19.0 19.0 19.0]


In [24]:
# Making arrays for training text and target author
txt_list = load_data(txt = date_integ[:,0])
#print(txt_list[0])
np_txt = np.array(txt_list)
print(np_txt)

tgt_list = date_integ[:,1]
#print(tgt_list)
np_tgt_list = tf.keras.utils.to_categorical(tgt_list)
#print(np_tgt_list)

[[21344 12402 12289 ...     0     0     0]
 [26397 12363 12425 ...     0     0     0]
 [12364 12289 20013 ...     0     0     0]
 ...
 [39640 23822 23665 ...     0     0     0]
 [12365 12398 12405 ...     0     0     0]
 [12381 12428 12364 ... 12398 12420 12358]]


In [25]:
# Creating character-level convolutional neural network model.

def create_model(embed_size=128, max_length=100, filter_sizes=(2, 3, 4, 5), filter_num=64):
    inp = tf.keras.layers.Input(shape=(max_length,))
    emb = tf.keras.layers.Embedding(0xffff, embed_size)(inp)
    emb_ex = tf.keras.layers.Reshape((max_length, embed_size, 1))(emb)
    convs = []
    for filter_size in filter_sizes:
        conv = tf.keras.layers.Convolution2D(filter_num, (filter_size, embed_size), activation="relu")(emb_ex)
        pool = tf.keras.layers.MaxPooling2D(pool_size=(max_length - filter_size + 1,1))(conv)
        convs.append(pool)
    convs_merged = tf.keras.layers.concatenate(convs, axis=-1)
    #convs_merged = tf.keras.layers.concatenate(convs, mode='concat')
    reshape = tf.keras.layers.Reshape((filter_num * len(filter_sizes),))(convs_merged)
    fc1 = tf.keras.layers.Dense(64, activation="relu")(reshape)
    bn1 = tf.keras.layers.BatchNormalization()(fc1)
    do1 = tf.keras.layers.Dropout(0.5)(bn1)
    fc2 = tf.keras.layers.Dense(20, activation='sigmoid')(do1)
    model = Model(inputs=inp, outputs=fc2)
    return model

In [45]:
# Training the model.
model_file = r"" #モデルの保存先

def train(inputs, targets, batch_size=100, epochs=50, max_length=100, model_filepath=model_file, learning_rate=0.001):
    start = learning_rate
    stop = learning_rate * 0.01
    learning_rates = np.linspace(start, stop, epochs)

    model = create_model(max_length=max_length)
    optimizer = Adam(lr=learning_rate)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()

    target = os.path.join('/tmp', 'weights.*.hdf5')
    files = [(f, os.path.getmtime(f)) for f in glob.glob(target)]
    if len(files) != 0:
        latest_saved_model = sorted(files, key=lambda files: files[1])[-1]
        model.load_weights(latest_saved_model[0])
    
    # Logging file for each epoch
    csv_logger_file = '/tmp/clcnn_training.log'
    # Checkpoint model for each epoch
    #checkpoint_filepath = "/tmp/weights.{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5"
    model.fit(inputs, targets,
              epochs=epochs,
              batch_size=batch_size,
              verbose=1,
              validation_split=0.1,
              shuffle=True,
              callbacks=[
                LearningRateScheduler(lambda epochs: learning_rates[epochs]),
                #CSVLogger(csv_logger_file),
                #ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, save_best_only=True, save_weights_only=False, monitor='val_acc')
              ])

    model.save("model_e50.h5")

In [29]:
train(np_txt, np_tgt_list)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 128)     8388480     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 100, 128, 1)  0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 99, 1, 64)    16448       ['reshape[0][0]']                
                                                                                              

c:\Users\yuta-negoro\anaconda3\envs\sakusya_suitei\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1373/1373 [==============================] - 99s 72ms/step - loss: 1.4303 - accuracy: 0.5844 - val_loss: 3.4527 - val_accuracy: 0.1555 - lr: 0.0010
Epoch 2/50
1373/1373 [==============================] - 99s 72ms/step - loss: 0.9943 - accuracy: 0.7054 - val_loss: 4.5576 - val_accuracy: 0.1155 - lr: 9.7980e-04
Epoch 3/50
1373/1373 [==============================] - 99s 72ms/step - loss: 0.8357 - accuracy: 0.7503 - val_loss: 3.2735 - val_accuracy: 0.1987 - lr: 9.5959e-04
Epoch 4/50
1373/1373 [==============================] - 99s 72ms/step - loss: 0.7315 - accuracy: 0.7796 - val_loss: 4.5118 - val_accuracy: 0.1416 - lr: 9.3939e-04
Epoch 5/50
1373/1373 [==============================] - 101s 74ms/step - loss: 0.6482 - accuracy: 0.8041 - val_loss: 5.3036 - val_accuracy: 0.0896 - lr: 9.1918e-04
Epoch 6/50
1373/1373 [==============================] - 99s 72ms/step - loss: 0.5809 - accuracy: 0.8231 - val_loss: 4.0482 - val_accuracy: 0.1966 - lr: 8.9898e-04
Epoch 7/50
1373/1373 [==============

In [30]:
# Encodes the raw_txt
def text_encoding(raw_txt):
    txt = [ord(x) for x in str(raw_txt).strip()]
    txt = txt[:100]
    if len(txt) < 100:
        txt += ([0] * (100 - len(txt)))
    return txt



# Predict
def predict(comments, model_filepath="model.h5"):
    model = load_model(model_filepath)
    ret = model.predict(comments)
    return ret


In [34]:
def author_data_integ_test():
    #作者ごとに
    for i in range(0,len(sakusya_list)):
        marge_csv = []
        sakusya_name = sakusya_list[i]
        date_path = ""
        date_path = os.path.join(aozora_dir, sakusya_name)
        date_path = os.path.join(date_path,"テストデータ")
        date_list = os.listdir(date_path)
        #print(date_list)
        #テキストの数だけ
        integ_np = np.array([["author", "line"]])
        for file in date_list:
            if "integ.csv" in file:
                pass
            else:
                date_path2 = os.path.join(date_path,file)
                #学習データ/iii/csv
                csv_dir = os.path.join(date_path2,"csv")
                csv_file = os.listdir(csv_dir)
                csv_file2 = csv_file[0]
                if "txt" in csv_file:
                    csv_file2 = csv_file[1] 
                #integ_np = np.array([["author", "line"]])
                csv_file_dir = os.path.join(csv_dir,csv_file2)

                pds_data = pds.read_csv(csv_file_dir,index_col=0)
                pds_data = pds_data.dropna()
                np_data = np.array(pds_data.iloc[:,[0,2]])
                out = [j for j in range(len(np_data)) if '-----------' in str(np_data[j,1])]
                if not out: out = [1]
                hyphen_pos = int(out[len(out) - 1])

                last_20 = len(np_data) - 20

                np_data = np_data[hyphen_pos+1:last_20,:]
                integ_np = np.vstack((integ_np, np_data))
                #print(pds_data)
                #marge_csv.append(pds_data)
        integ_pds = pds.DataFrame(integ_np[1:,:], columns=integ_np[0,:])
        integ_pds.to_csv(date_path+ "/" + sakusya_name + '_integ.csv', quoting=csv.QUOTE_ALL)   
    
            #前5行と後ろ20行を削除
        #integ_pds = pds.concat(marge_csv, ignore_index=True)    
        #integ_pds.to_csv(date_path+ "/" + sakusya_name + '_integ.csv', quoting=csv.QUOTE_ALL)

In [65]:
#テストデータの前処理
#作者の数だけ
for i in range(0,len(sakusya_list)):
    sakusya_name = sakusya_list[i]
    print(sakusya_name)
    date_path = os.path.join(aozora_dir, sakusya_name)
    #作者/テストデータ/
    date_path = os.path.join(date_path,"テストデータ")
    date_list = os.listdir(date_path)

    #作者/テストデータ/48909
    test_date_dir = os.path.join(date_path,date_list[0])
    test_date_list = os.listdir(test_date_dir)
    test_date = test_date_list[0]
    for j in range(0,len(test_date_list)):
        convert_sjis_to_utf8(test_date_list,test_date_dir,sakusya_name)
        data_cleanse(test_date_list,test_date_dir,sakusya_name)



内田 魯庵
南部 修太朗
古川 緑波
土田 耕平
大倉 てる子
宮原 晃一郎
小栗 虫太郎
山之口 獏
新渡戸 稲造
桑原 隲蔵
正岡 容
泉 鏡太郎
渡辺 温
竹久 夢二
萩原 朔太郎
葛西 善蔵
蘭 郁二郎
野上 豊一郎
野口 雨情
高浜 虚子


In [70]:
author_data_integ_test()

In [44]:
target_author = sakusya_list
#テストデータの文一行ずつ分類モデルで作者を予想する
#作者の数だけ
sakusya_count = []
sakusya_num = int(len(sakusya_list))
sakusya_count = [0] * sakusya_num
sakuisya_suitei_rank = []
seikai_num = 0

header = ["テストデータの作者","推定した作者1位","推定した作者2位","推定した作者3位","合否"]
#結果をcsvで出力
with open(aozora_dir + "/result.csv",'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    #print(sakusya_count)
    for i in range(0,len(sakusya_list)):
        #カウント初期化
        sakusya_count = [0] * sakusya_num
        date_path = ""
        sakusya_name = sakusya_list[i]
        print(sakusya_name)
        date_path = os.path.join(aozora_dir, sakusya_name)
        #作者/テストデータ/
        date_path = os.path.join(date_path,"テストデータ")
        date_list = os.listdir(date_path)
        for j in date_list:
            if "integ.csv" in j:
                date_path = os.path.join(date_path,j)
                csv_input = pds.read_csv(filepath_or_buffer=date_path,encoding="utf-8",sep=",")
                #print(csv_input.values)
                for l in range(0,len(csv_input.values)):
                    #print(csv_input.values[l,2])
                    txt = csv_input.values[l,2]
                    txt = str(txt)
                    txt = text_encoding(txt)
                    #print(txt)
                    predict_result = predict(np.array([txt]), model_filepath=model_file)
                    pds_predict_result = pds.DataFrame(predict_result, columns=target_author)
                    #print(pds_predict_result)
                    #一番高い類似度の作者をテストデータの作者とする それを一文ずつ行う
                    max_ruigido = 0
                    #一番類似度が高かった作者 0 ~ 19
                    max_sakusya_num = 0
                    for num in range(0,len(pds_predict_result.values[0])):
                        now_r = pds_predict_result.values[0][num]
                        if now_r > max_ruigido:
                            max_ruigido = now_r
                            max_sakusya_num = num
                    #ここでカウントを保存
                    sakusya_count[max_sakusya_num] += 1
        print("モデルの作者推定の結果")
        print(sakusya_count)
        sort_list = sorted(set(sakusya_count))
        ans = sakusya_count.index(sort_list[-1])
        first = ans
        ans = sakusya_count.index(sort_list[-2])
        second = ans
        ans = sakusya_count.index(sort_list[-3])
        third = ans
        if first == i:
            print("正解")
            seikai_num += 1
            gouhi = "正解"
            print("一文ずつの精度", int(( int(sort_list[-1])/ int(len(csv_input.values))) * 100), "%")
        else:
            print("不正解")
            gouhi = "不正解"

        writer.writerow([sakusya_name,sakusya_list[first],sakusya_list[second],sakusya_list[third],gouhi])
print("正答率",seikai_num,"/",sakusya_num)
seido = seikai_num / sakusya_num


    #txt = text_encoding(s)
    #print(txt)
    #predict_result = predict(np.array([txt]), model_filepath=model_file)
    #pds_predict_result = pds.DataFrame(predict_result, columns=target_author)
    #print(pds_predict_result)

内田 魯庵
1/1 [==============================] - 0s 88ms/step
モデルの作者推定の結果
[13, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 4]
正解
一文ずつの精度 65 %
南部 修太朗
1/1 [==============================] - 0s 87ms/step
モデルの作者推定の結果
[1, 29, 2, 0, 0, 0, 2, 0, 0, 1, 0, 5, 0, 7, 2, 6, 0, 7, 0, 0]
正解
一文ずつの精度 46 %
古川 緑波
1/1 [==============================] - 0s 84ms/step
モデルの作者推定の結果
[0, 4, 7, 0, 0, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1, 8, 0, 3, 0, 0]
不正解
土田 耕平
1/1 [==============================] - 0s 123ms/step
モデルの作者推定の結果
[0, 0, 2, 83, 5, 2, 3, 0, 0, 0, 4, 11, 4, 3, 0, 0, 0, 0, 1, 0]
正解
一文ずつの精度 70 %
大倉 てる子
1/1 [==============================] - 0s 84ms/step
モデルの作者推定の結果
[4, 2, 7, 1, 228, 4, 32, 2, 3, 1, 19, 19, 25, 2, 6, 9, 16, 0, 3, 9]
正解
一文ずつの精度 58 %
宮原 晃一郎
1/1 [==============================] - 0s 102ms/step
モデルの作者推定の結果
[1, 21, 14, 4, 1, 18, 22, 3, 5, 4, 5, 10, 2, 12, 5, 16, 2, 23, 3, 2]
不正解
小栗 虫太郎
1/1 [==============================] - 0s 88ms/step
モデルの作者推定の結果
[0, 0, 0, 0, 0, 0, 12, 0, 1, 0, 0, 0, 0, 0, 1, 